# Diabetic retinopathy classification

## 02 Selecting equal samples datasets for each eye/class

## Imports

In [9]:
import torch
from PIL import Image, ImageOps
import pickle
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import Dataset, DataLoader
import time
import copy
import pandas as pd
import os
from functions import *

## Loading image names and respective labels

In [2]:
outputs = pd.read_csv("d:/project_data/trainLabels.csv")

In [3]:
outputs.columns

Index(['image', 'level'], dtype='object')

## Inspecting number of samples for each class

In [4]:
outputs.level.value_counts()

0    25810
2     5292
1     2443
3      873
4      708
Name: level, dtype: int64

## Preparing containers for paths storage

In [8]:
num_classes = 5
img_paths_right = [ [] for _ in range (num_classes)]
img_paths_left = [ [] for _ in range (num_classes)]

proceed = 10
level_status = [0 for i in range(10)]
dir_path = 'd:\\project_data\\train_1'
image_files=os.listdir(dir_path)

## Selector checks duplicates

In [10]:
data = selector(image_files, outputs)

index = data.index
number_of_rows = len(index)
number_of_rows

35124

## Seaparting image paths for each <eye, class> based on lowest class distribution

In [ ]:
row = 0

maxi = 350 # number of samples per class per eye

while proceed:
    try:
        #print(data["level"].value_counts())
        #print(len(image_files)
        level = int(data["level"][row])
        
        abs_path = os.path.join(dir_path, data['image'][row]) + '.jpeg'
        if 'left' in abs_path:
            if level_status[level] < maxi:
                img_paths_left[level].append(abs_path)
                level_status[level]+=1 
            elif level_status[level] == maxi:
                proceed-=1
                level_status[level]+=1            
        elif 'right' in abs_path:
            if level_status[level+5] < maxi:
                img_paths_right[level].append(abs_path)
                level_status[level+5]+=1   
            elif level_status[level+5] == maxi:
                proceed-=1
                level_status[level+5]+=1                        
    except:
        pass
    
    if row%100==0:
        print(f'{row}/{maxi*num_classes*2}') 
        print(level_status)
    row+=1
    if row > number_of_rows:
        proceed = 0

## Save paths in files

In [ ]:
import pickle

lefts = f'nochers_5x{maxi}_left'
rights = f'nochers_5x{maxi}_right'

with open(lefts, 'wb') as f:
    pickle.dump(img_paths_left, f)
with open(rights, 'wb') as f:
    pickle.dump(img_paths_right, f)